In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, f1_score, auc, precision_score, recall_score
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image
from collections import Counter
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import xlrd
import openpyxl
import copy

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# define training and testing data directories
source_path = 'F:/Original/All FR/train'

class_names = [folder for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]
num_classes = len(class_names)

file_list = os.listdir(source_path)
model_dir = 'Convnext_pretrained_younas.pt'

output_path = 'F:/Anonymized/EDI/Pix/Face_Recognition'

save_roc_dir = 'F:/Results_Final/ROC/ROC Plots Mouth_Masked/'
excel_file_path = 'F:/Results_Final/Excel_Sheets/EDI_All_FR_Results.xlsx'

In [ ]:
# Define a new transform with additional data augmentations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

In [ ]:
model = models.convnext_base(pretrained=True)
model.classifier[2]=nn.Linear(1024,num_classes)

model.load_state_dict(torch.load('Convnext_pretrained_younas.pt'))
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.001)

In [ ]:
# We will try a fixed q in each iteration
q=95

In [ ]:
#Setting Parameter values

#For Simple Blur
blur_kernel_size = 40

#For Simple Pixelate
pixel_size = 20


#for DP
#create_dp_pixelated_image(x, b, m, eps)
# we need x which is the input image, b which is the block size, the higher (after a certain value) the more noise,
#lower before a certain value does not automatically reduce noise if a very small eps is used

#Larger m more neighbors disnguishability hence more noise, lesser eps more noise

#For DP Pix
b = 32
m = 16
eps = 100


#create_dp_blurred_image(x, b0, m, eps, k, sigma)
# x, m, and eps are the same
#whereas, we need to use a different b (b0) for pixelation and DP pixelation, which would reduce the noise and just pixelate it
#K is kernel size for gaussian blur. Height and width of the Gaussian kernel
#SD of Gaussian blur. Larger sigma means more blurring, the paper did not use this, so we are using 0, in both cases

b0 = 32 #recommended half of the original b
k = 9
sigma = 0

In [ ]:
# def optimize_gradients(gradients, q):
#     grad = gradients.cpu().squeeze().data.detach().cpu().numpy()
#     q_local = get_quartiles(grad, q)
#     idx = np.where(grad < q_local)
#     total = len(idx[2])+len(idx[1])+len(idx[2])
#     print(total)
#     idx_up = np.where(grad >= q_local)
#     grad[idx] = 0
#     grad[idx_up] = 1
#     grad = np.transpose(grad, (1, 2, 0))
#     trans1 = transforms.ToTensor()
#     grad = trans1(grad)
#     grad = torch.unsqueeze(grad, 0)
#     return grad

In [ ]:
start_time = time.time()
new_batch_size = 1
new_test_path = 'F:/Anonymized/EDI/Pix/Face_Recognition/Iteration_68'
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
prev_acc = final_acc
print(prev_acc*100)

end_time = time.time()
acc_time = end_time - start_time
print(f"\nAcc execution time: {acc_time} seconds")

In [ ]:
start_time = time.time()

# If the number of iterations crashed your system, try the iterations in batches (batch of 5)
for itera in range(69,72):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)

        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"

        class_label = labels[i]
        class_subfolder = class_names[class_label]            

        # Create subfolder based on the class name
    #         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True

        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)


        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data
            
#             perturbed = create_dp_blurred_image(x, b0, m, eps, k, sigma)
#             perturbed = create_dp_pixelated_image(x, b, m, eps)

    # Call different perturbation functions here
#             perturbed = create_blurred_image(x, blur_kernel_size)
            perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_dp_blurred_image(x, b0, m, eps, k, sigma)
            


            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients

            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, 100-q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, 100-q)


    torch.cuda.empty_cache()
    
    our_test_loader = create_test_loader(iteration_out_path, batch_size=1)
    accuracy, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
    print(f"\nAccuracy: {accuracy*100} %")

end_time = time.time()
Anon_execution_time = end_time - start_time
print(f"\nAnon Time: {Anon_execution_time} seconds")
print(f"All images processed for iteration number {itera}.")

Pix with BS 20

Accuracy: 49.72692517749863 %

Anon Time: 1655.7662649154663 seconds
All images processed for iteration number 2.

Accuracy: 21.545603495357728 %

Anon Time: 1424.089788198471 seconds
All images processed for iteration number 5.

Accuracy: 12.424904423812125 %

Anon Time: 1391.1427669525146 seconds
All images processed for iteration number 8.

Accuracy: 8.083014746040416 %

Anon Time: 1365.8819558620453 seconds
All images processed for iteration number 11.

Accuracy: 6.1714909885308575 %

Anon Time: 1418.8250737190247 seconds
All images processed for iteration number 14.

Accuracy: 5.488803932277444 %

Anon Time: 1422.5699796676636 seconds
All images processed for iteration number 17.

Accuracy: 4.478427089022392 %

Anon Time: 1402.0778336524963 seconds
All images processed for iteration number 20.

Accuracy: 3.9595849262697977 %

Anon Time: 1411.1460461616516 seconds
All images processed for iteration number 23.

Accuracy: 3.46805024576734 %

Anon Time: 1434.4932234287262 seconds
All images processed for iteration number 26.

Accuracy: 2.9219006007646096 %

Anon Time: 1422.658697128296 seconds
All images processed for iteration number 29.

Accuracy: 2.184598580010923 %

Anon Time: 1448.3849272727966 seconds
All images processed for iteration number 32.

Accuracy: 2.3757509557618786 %

Anon Time: 1441.4864106178284 seconds
All images processed for iteration number 35.

Accuracy: 2.3211359912616056 %

Anon Time: 1459.0054905414581 seconds
All images processed for iteration number 38.

Accuracy: 2.457673402512288 %

Anon Time: 1429.6848278045654 seconds
All images processed for iteration number 41.

Accuracy: 2.020753686510104 %

Anon Time: 1447.451754808426 seconds
All images processed for iteration number 44.

Accuracy: 1.8569087930092845 %

Anon Time: 1438.3847453594208 seconds
All images processed for iteration number 47.

Accuracy: 1.774986346258875 %

Anon Time: 1462.4705891609192 seconds
All images processed for iteration number 50.

Accuracy: 1.5019115237575096 %

Anon Time: 1414.225788116455 seconds
All images processed for iteration number 53.

Accuracy: 1.3926815947569635 %

Anon Time: 1408.6027116775513 seconds
All images processed for iteration number 56.

Accuracy: 1.2834516657564172 %

Anon Time: 1445.8384294509888 seconds
All images processed for iteration number 59.

Accuracy: 1.2561441835062808 %

Anon Time: 1410.9528658390045 seconds
All images processed for iteration number 62.

Accuracy: 1.3107591480065537 %

Anon Time: 1408.3483335971832 seconds
All images processed for iteration number 65.

Accuracy: 1.1469142545057347 %

Anon Time: 1429.120777606964 seconds
All images processed for iteration number 68.

Pix with BS 15

Accuracy: 53.54997269251774 %

Anon Time: 1641.5903458595276 seconds
All images processed for iteration number 2.


Accuracy: 24.822501365374112 %

Anon Time: 1475.218293428421 seconds
All images processed for iteration number 5.

Accuracy: 14.44565811032223 %

Anon Time: 1510.5347275733948 seconds
All images processed for iteration number 8.

Accuracy: 9.503003823047514 %

Anon Time: 1438.1695866584778 seconds
All images processed for iteration number 11.

Accuracy: 8.301474604041507 %

Anon Time: 1434.8055348396301 seconds
All images processed for iteration number 14.


Accuracy: 7.045330420535227 %

Anon Time: 1665.0537581443787 seconds
All images processed for iteration number 17.

Accuracy: 7.0999453850355 %

Anon Time: 1404.3785421848297 seconds
All images processed for iteration number 20.

Accuracy: 5.516111414527581 %

Anon Time: 1659.36869931221 seconds
All images processed for iteration number 23.

Accuracy: 5.133806663025669 %

Anon Time: 1733.8373816013336 seconds
All images processed for iteration number 26.

Accuracy: 4.423812124522119 %

Anon Time: 1804.1841077804565 seconds
All images processed for iteration number 29.

Accuracy: 3.6865101037684327 %

Anon Time: 1443.3368220329285 seconds
All images processed for iteration number 32.

Accuracy: 3.9595849262697977 %

Anon Time: 1430.9130320549011 seconds
All images processed for iteration number 35.


Accuracy: 3.358820316766794 %

Anon Time: 1452.633713722229 seconds
All images processed for iteration number 38.

Accuracy: 3.1676679410158384 %

Anon Time: 1446.1268725395203 seconds
All images processed for iteration number 41.

Accuracy: 3.003823047515019 %

Anon Time: 1418.2089540958405 seconds
All images processed for iteration number 44.

Accuracy: 2.4030584380120152 %

Anon Time: 1415.6000699996948 seconds
All images processed for iteration number 47.





Anon Time: 1705.145141839981 seconds
All images processed for iteration number 2.


Accuracy: 48.41616602949208 %

Anon Time: 1606.145141839981 seconds
All images processed for iteration number 5.

Accuracy: 30.638995084653192 %

Anon Time: 1528.3995959758759 seconds
All images processed for iteration number 8.

Accuracy: 19.79792463134899 %

Anon Time: 1475.8778047561646 seconds
All images processed for iteration number 11.

missed one

Accuracy: 8.137629710540688 %

Anon Time: 1445.648427248001 seconds
All images processed for iteration number 17.

Accuracy: 5.953031130529765 %

Anon Time: 1435.8225164413452 seconds
All images processed for iteration number 20.

Accuracy: 4.560349535772802 %

Anon Time: 1480.136723279953 seconds
All images processed for iteration number 23.

Accuracy: 3.3042053522665213 %

Anon Time: 1453.2622804641724 seconds
All images processed for iteration number 26.

Accuracy: 2.7307482250136537 %

Anon Time: 1471.7750742435455 seconds
All images processed for iteration number 29.

Accuracy: 2.2938285090114694 %

Anon Time: 1456.1509764194489 seconds
All images processed for iteration number 32.

Accuracy: 1.774986346258875 %

Anon Time: 1460.2456846237183 seconds
All images processed for iteration number 35.


Accuracy: 1.5292190060076463 %

Anon Time: 1433.1150388717651 seconds
All images processed for iteration number 38.


Accuracy: 1.1742217367558712 %

Anon Time: 1447.6232042312622 seconds
All images processed for iteration number 41.

Accuracy: 0.8738394320043691 %

Anon Time: 1438.8332269191742 seconds
All images processed for iteration number 44.

Accuracy: 98.68924085199345 %

Anon Time: 1694.9894626140594 seconds
All images processed for iteration number 2.

Accuracy: 91.88967777170944 %

Anon Time: 1662.7095577716827 seconds
All images processed for iteration number 5.

Accuracy: 80.44784270890224 %

Anon Time: 1647.600581407547 seconds
All images processed for iteration number 8.

Accuracy: 67.47678864008738 %

Anon Time: 1579.717096567154 seconds
All images processed for iteration number 11.


Accuracy: 54.94265428727472 %

Anon Time: 1555.2830095291138 seconds
All images processed for iteration number 14.

Accuracy: 44.15619879847078 %

Anon Time: 1508.8828449249268 seconds
All images processed for iteration number 17.

Accuracy: 35.28126706717641 %

Anon Time: 1478.4752361774445 seconds
All images processed for iteration number 20.

Accuracy: 27.826324412889132 %

Anon Time: 1418.6979348659515 seconds
All images processed for iteration number 23.





Accuracy: 97.21463681048607 %

Anon Time: 1642.7934617996216 seconds
All images processed for iteration number 5.

Accuracy: 92.32659748771164 %

Anon Time: 1667.3688173294067 seconds
All images processed for iteration number 8.

Accuracy: 85.14472965592572 %

Anon Time: 1639.1201922893524 seconds
All images processed for iteration number 11.

Accuracy: 76.54287274713272 %

Anon Time: 1595.39142203331 seconds
All images processed for iteration number 14.

Accuracy: 67.77717094483889 %

Anon Time: 1615.782452583313 seconds
All images processed for iteration number 17.

Accuracy: 59.14800655379574 %

Anon Time: 1505.7750158309937 seconds
All images processed for iteration number 20.

Accuracy: 51.88421627525942 %

Anon Time: 1514.0334422588348 seconds
All images processed for iteration number 23.

Accuracy: 44.72965592572365 %

Anon Time: 1507.9489579200745 seconds
All images processed for iteration number 26.

Accuracy: 38.31239759694156 %

Anon Time: 1482.005931377411 seconds
All images processed for iteration number 29.

Accuracy: 32.41398143091207 %

Anon Time: 1462.3578870296478 seconds
All images processed for iteration number 32.

Accuracy: 27.74440196613872 %

Anon Time: 1439.2906155586243 seconds
All images processed for iteration number 35.

Accuracy: 23.75750955761879 %

Anon Time: 1410.6026022434235 seconds
All images processed for iteration number 38.

Accuracy: 20.562534134352813 %

Anon Time: 1410.4384293556213 seconds
All images processed for iteration number 41.

Accuracy: 17.149098853085746 %

Anon Time: 1449.3562803268433 seconds
All images processed for iteration number 44.


Accuracy: 14.74604041507373 %

Anon Time: 1400.3454177379608 seconds
All images processed for iteration number 47.

Accuracy: 13.462588749317314 %

Anon Time: 411.9242467880249 seconds
All images processed for iteration number 48.

Accuracy: 98.03386127799017 %

Anon Time: 1835.5235884189606 seconds
All images processed for iteration number 2.

Accuracy: 88.72200983069362 %

Anon Time: 1732.3147201538086 seconds
All images processed for iteration number 5.

Accuracy: 72.61059530311306 %

Anon Time: 1725.6176888942719 seconds
All images processed for iteration number 8.

Accuracy: 56.690333151283454 %

Anon Time: 2178.500242471695 seconds
All images processed for iteration number 11.

Accuracy: 43.96504642271983 %

Anon Time: 1521.6785006523132 seconds
All images processed for iteration number 14.

Accuracy: 33.451665756417256 %

Anon Time: 1441.194378376007 seconds
All images processed for iteration number 17.

Accuracy: 26.788640087383943 %

Anon Time: 1729.4973013401031 seconds
All images processed for iteration number 20.

Accuracy: 20.75368651010377 %

Anon Time: 1549.8581056594849 seconds
All images processed for iteration number 23.

Accuracy: 16.193336974330965 %

Anon Time: 1449.5039274692535 seconds
All images processed for iteration number 26.



Blur

Accuracy: 94.6204259967231 %

Anon Time: 1670.2861230373383 seconds
All images processed for iteration number 2.

Accuracy: 76.59748771163298 %

Anon Time: 1605.4640808105469 seconds
All images processed for iteration number 5.

Accuracy: 54.833424358274165 %

Anon Time: 1535.8226952552795 seconds
All images processed for iteration number 8.

Accuracy: 36.6193336974331 %

Anon Time: 1472.0906715393066 seconds
All images processed for iteration number 11.

Accuracy: 25.53249590387766 %

Anon Time: 1443.523648262024 seconds
All images processed for iteration number 14.

Accuracy: 17.476788640087385 %

Anon Time: 1416.9053795337677 seconds
All images processed for iteration number 17.

Accuracy: 12.479519388312397 %

Anon Time: 1397.1633858680725 seconds
All images processed for iteration number 20.

iteration 19 does it

Pix

Accuracy: 68.37793555434189 %

Anon Time: 1653.597927570343 seconds
All images processed for iteration number 2.

Accuracy: 37.38394320043692 %

Anon Time: 1471.2509217262268 seconds
All images processed for iteration number 5.

Accuracy: 25.805570726379027 %

Anon Time: 1417.1616575717926 seconds
All images processed for iteration number 8.

LAST 3 ARE MISSING


DP Blur

Accuracy: 60.26761332605134 %

Anon Time: 1795.316531419754 seconds
All images processed for iteration number 2.

missed one here

Accuracy: 14.145275805570726 %

Anon Time: 1477.6396446228027 seconds
All images processed for iteration number 8.

Accuracy: 7.755324959038777 %

Anon Time: 1404.6242561340332 seconds
All images processed for iteration number 11.

Accuracy: 5.652648825778264 %

Anon Time: 1392.6372911930084 seconds
All images processed for iteration number 14.

Accuracy: 3.6865101037684327 %

Anon Time: 1398.7609040737152 seconds
All images processed for iteration number 17.

Accuracy: 3.2495903877662475 %

Anon Time: 1420.8439662456512 seconds
All images processed for iteration number 20.

Accuracy: 2.2665210267613327 %

Anon Time: 1436.6383368968964 seconds
All images processed for iteration number 23.

Accuracy: 2.020753686510104 %

Anon Time: 1427.342900276184 seconds
All images processed for iteration number 26.

Accuracy: 1.9388312397596943 %

Anon Time: 1448.3249609470367 seconds
All images processed for iteration number 29.


Accuracy: 1.7476788640087382 %

Anon Time: 1415.4095394611359 seconds
All images processed for iteration number 32.


Accuracy: 1.6111414527580556 %

Anon Time: 1442.3765335083008 seconds
All images processed for iteration number 35.


Accuracy: 1.3107591480065537 %

Anon Time: 1449.046518087387 seconds
All images processed for iteration number 38.


Accuracy: 0.9830693610049154 %

39 is also 0.98

Anon Time: 1450.2956638336182 seconds
All images processed for iteration number 41.





DP Pix

Plus 100 seconds

Anon Time: 1693.9437828063965 seconds
All images processed for iteration number 2.

Time for 3, 4 and 5 needs to be done again, or manually inferenced

Anon Time: 1442.4036922454834 seconds
All images processed for iteration number 8.

Anon Time: 1428.0455169677734 seconds
All images processed for iteration number 11.

Anon Time: 1439.1756038665771 seconds
All images processed for iteration number 14.

Anon Time: 1427.472996711731 seconds
All images processed for iteration number 17.

Accuracy: 3.194975423265975 %

Anon Time: 1441.0913925170898 seconds
All images processed for iteration number 23.

Accuracy: 2.4030584380120152 %

Anon Time: 1443.2411394119263 seconds
All images processed for iteration number 26.

Accuracy: 2.2938285090114694 %

Anon Time: 1406.3963334560394 seconds
All images processed for iteration number 29.

Accuracy: 2.1572910977607864 %

Anon Time: 1431.9816498756409 seconds
All images processed for iteration number 32.

Iteration 30 achieves it


Blur

Accuracy: 99.64500273074822 %

Anon Time: 1677.3597085475922 seconds
All images processed for iteration number 2.

Accuracy: 97.21463681048607 %

Anon Time: 1629.5300686359406 seconds
All images processed for iteration number 5.

Accuracy: 92.32659748771164 %

Anon Time: 1660.037616968155 seconds
All images processed for iteration number 8.

Accuracy: 85.14472965592572 %

Anon Time: 1583.4810528755188 seconds
All images processed for iteration number 11.

Accuracy: 76.54287274713272 %

Anon Time: 1589.7678318023682 seconds
All images processed for iteration number 14.

Accuracy: 67.77717094483889 %

Anon Time: 1572.1026682853699 seconds
All images processed for iteration number 17.

Accuracy: 59.14800655379574 %

Anon Time: 1499.0802021026611 seconds
All images processed for iteration number 20.

Accuracy: 51.88421627525942 %

Anon Time: 1498.8100275993347 seconds
All images processed for iteration number 23.

Accuracy: 44.72965592572365 %

Anon Time: 1485.097520828247 seconds
All images processed for iteration number 26.

Accuracy: 38.31239759694156 %

Anon Time: 1472.33660197258 seconds
All images processed for iteration number 29.

Accuracy: 32.41398143091207 %

Anon Time: 1453.926073551178 seconds
All images processed for iteration number 32.

Accuracy: 27.74440196613872 %

Anon Time: 1436.5961413383484 seconds
All images processed for iteration number 35.

Accuracy: 23.75750955761879 %

Anon Time: 1395.828239440918 seconds
All images processed for iteration number 38.

Accuracy: 20.562534134352813 %

Anon Time: 1399.549635887146 seconds
All images processed for iteration number 41.

Iteration 43

Accuracy: 18.405243036592026 %